In [1]:
from utils import computeR2
from utils import computeR2_train
from utils import computeR2_train_self
from utils import computeR2_unlabel
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist, squareform
import pandas as pd

In [4]:
import os 
os.chdir('/Users/haoliu/Documents/GitHub/Active-Learning-Using-various-representations')

In [2]:
pwd

'/Users/haoliu/Documents/GitHub/Active-Learning-Using-various-representations'

In [3]:
from utils import get_input
from utils import get_input_all
from utils import normalized_data
from Save_data import save_data
from Plot_performance import plot_performance
import pickle
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')
from utils import feature_selection

In [4]:
# Hyperparameters
numDataTotal = 509
labeledPoolN = 10
batchSz = 1
nAccs = (numDataTotal-labeledPoolN)//batchSz
RepeatTimes = 1
fs_score=0.98
freq=10
Alg='GSx_fs'
# Get the input files
myfile =r'Data/CombinedPSP.csv'
# X, y = get_input(myfile)
# If combine feature selection
X, y = get_input_all(myfile)

In [6]:
R2Smooth = []
MSEsmooth = []
MAEsmooth = []
Infosmooth = []
R2_train = []
R2_trainS = []
SelectData = []

for rt in tqdm(range(RepeatTimes)):
    np.random.seed(rt)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=rt)
    dataPool = pd.concat([X_train, y_train], axis=1)
    # Reset the index to ensure the indices are continuous integers
    # dataPool = dataPool.reset_index(drop=True)
    SelectIdx=np.random.choice(dataPool.index, labeledPoolN, replace=False)
    dataPoolL = dataPool.iloc[SelectIdx, :]
    dataPool = dataPool.drop(SelectIdx)
    # dataPool = dataPool.reset_index(drop=True)
    
    data = pd.concat([X_train, y_train], axis=1)
    # dataPool = dataPool.reset_index(drop=True)
    distX = squareform(pdist(data.iloc[:,0:-1]))

    Idx = []
    Idx = SelectIdx.tolist()
    #Idx.append(SelectIdx)
    idsTest=np.arange(0,len(y_train))
    idsTest=np.delete(idsTest,Idx)
    print(f'Idx: {len(Idx)}')
    print(f'idsTest: {idsTest.shape}')#here

    R2Res = np.empty((0,1), float)
    MSERes = np.empty((0,1), float)
    MAERes = np.empty((0,1), float)
    InfoRes = np.empty((0,1), float)
    R2Res_t = np.empty((0,1), float)
    R2Res_tS = np.empty((0,1), float)

    R2, Model, MSEstart, MAEstart = computeR2(dataPoolL, X_test, y_test, fs=True)
    R2Res = np.append(R2Res, R2, axis=0)
    MSERes = np.append(MSERes, MSEstart, axis=0)
    MAERes = np.append(MAERes, MAEstart, axis=0)
    Info = computeR2_unlabel(dataPool, dataPoolL, Model, fs=True)
    InfoRes = np.append(InfoRes, Info, axis=0)

    R2_t, Model, MSEstart_t,_ = computeR2_train(dataPoolL, X_train, y_train, fs=True)
    R2Res_t = np.append(R2Res_t, R2_t, axis=0)

    R2_tS, ModelS, MSEstart_tS,_ = computeR2_train_self(dataPoolL, fs=True)
    R2Res_tS = np.append(R2Res_tS, R2_tS, axis=0)

    # feature selection
    indices = feature_selection(dataPoolL.iloc[:, 0:-1],dataPoolL.iloc[:, -1], fs_score, 0, Alg)
    dataPoolL_fs = pd.concat([dataPoolL.iloc[:, 0:-1].iloc[:, indices],dataPoolL.iloc[:, -1]],axis=1)
    dataPool_fs = pd.concat([dataPool.iloc[:, 0:-1].iloc[:, indices], dataPool.iloc[:, -1]],axis=1)
    data_fs = pd.concat([data.iloc[:, 0:-1].iloc[:, indices], data.iloc[:, -1]],axis=1)

    # get model with fewer features
    # _, Model_fs, _, _ = computeR2(dataPoolL_fs, X_test.iloc[:, indices], y_test, fs=True)

    for i in range(10, 509):
#         print(i)
        
        distX = squareform(pdist(data_fs.iloc[:,0:-1]))
    
        # Idx = []
        # Idx = SelectIdx.tolist()
        #Idx.append(SelectIdx)
        # idsTest=np.arange(0,len(y_train))
        # idsTest=np.delete(idsTest,Idx)
    
        dist=distX[np.ix_(idsTest, np.array(Idx[0:i]))].min(axis=1)
        idx=np.argmax(dist)
        Idx.append(idsTest[idx])
        idsTest=np.delete(idsTest,idx)
        print(f'idx:{idx}')
        print(f'Idx: {len(Idx)}')
        print(f'idsTest: {idsTest.shape}')

        # not cancat with pd.series
        databatch_fs=dataPool_fs.iloc[idx,:].to_frame().T
        dataPool_fs=data_fs.iloc[idsTest,:]
        dataPoolL_fs = pd.concat([dataPoolL_fs, databatch_fs], axis=0)

        # revover features
        databatch=dataPool.iloc[idx,:].to_frame().T
        dataPool=data.iloc[idsTest,:]
        dataPoolL = pd.concat([dataPoolL, databatch], axis=0)

        cR2, Model, cMSE, cMAE = computeR2(dataPoolL, X_test, y_test, fs=True)#here
        R2Res = np.append(R2Res, cR2, axis=0)
        MSERes = np.append(MSERes, cMSE, axis=0)
        MAERes = np.append(MAERes, cMAE, axis=0)
        cInfo = computeR2_unlabel(dataPool, dataPoolL, Model, fs=True)
        InfoRes = np.append(InfoRes, cInfo, axis=0)

        cR2_t, Model, cMSEstart_t,_ = computeR2_train(dataPoolL, X_train, y_train, fs=True)
        R2Res_t = np.append(R2Res_t, cR2_t, axis=0)

        cR2_tS, ModelS, cMSEstart_tS,_ = computeR2_train_self(dataPoolL, fs=True)
        R2Res_tS = np.append(R2Res_tS, cR2_tS, axis=0)

        if i % freq == 0:

            # feature selection
            indices = feature_selection(dataPoolL.iloc[:, 0:-1],dataPoolL.iloc[:, -1], fs_score, 0, Alg)
            dataPoolL_fs = pd.concat([dataPoolL.iloc[:, 0:-1].iloc[:, indices],dataPoolL.iloc[:, -1]],axis=1)
            dataPool_fs = pd.concat([dataPool.iloc[:, 0:-1].iloc[:, indices], dataPool.iloc[:, -1]],axis=1)
            data_fs = pd.concat([data.iloc[:, 0:-1].iloc[:, indices], data.iloc[:, -1]],axis=1)

            # doesn't need to run the performance again?
            # get model with fewer features
            # _, Model_fs, _, _ = computeR2(dataPoolL_fs, X_test.iloc[:, indices], y_test, fs=True)

            # R2Res = np.append(R2Res, cR2, axis=0)
            # MSERes = np.append(MSERes, cMSE, axis=0)
            # MAERes = np.append(MAERes, cMAE, axis=0)
            # InfoRes = np.append(InfoRes, cInfo, axis=0)
            # R2Res_t = np.append(R2Res_t, cR2_t, axis=0)
            # R2Res_tS = np.append(R2Res_tS, cR2_tS, axis=0)

    R2Smooth.append(R2Res)
    MSEsmooth.append(MSERes)
    MAEsmooth.append(MAERes)
    Infosmooth.append(InfoRes)
    R2_train.append(R2Res_t)
    R2_trainS.append(R2Res_tS)
    SelectData.append(dataPoolL)

#     SelectData.append(dataPoolL)
R2Smooth = np.asarray(R2Smooth)
MSEsmooth = np.asarray(MSEsmooth)
MAEsmooth = np.asarray(MAEsmooth)
InfoSmooth = np.asarray(Infosmooth)
R2_train = np.asarray(R2_train)
R2_trainS = np.asarray(R2_trainS) 

  0%|          | 0/1 [00:00<?, ?it/s]

Idx: 10
idsTest: (1356,)
Features sorted by their score:
[(0.1431, 'd11'), (0.1402, 'd3'), (0.1222, 'd14'), (0.1141, 'd21'), (0.0857, 'd18'), (0.079, 'd19'), (0.0658, 'd7'), (0.0505, 'd16'), (0.0414, 'd1'), (0.0305, 'd13'), (0.0287, 'd2'), (0.0232, 'd5'), (0.022, 'd10'), (0.0208, 'd4'), (0.016, 'd15'), (0.0098, 'd9'), (0.0024, 'd20'), (0.0016, 'd6'), (0.0015, 'd12'), (0.0007, 'd8'), (0.0007, 'd17')]
idx:419
Idx: 11
idsTest: (1355,)
Features sorted by their score:
[(0.1428, 'd3'), (0.1415, 'd16'), (0.1231, 'd11'), (0.1149, 'd18'), (0.1075, 'd19'), (0.0788, 'd1'), (0.0472, 'd21'), (0.0385, 'd2'), (0.0324, 'd15'), (0.0305, 'd7'), (0.0295, 'd8'), (0.0225, 'd10'), (0.0206, 'd14'), (0.0196, 'd5'), (0.0195, 'd4'), (0.0102, 'd17'), (0.01, 'd9'), (0.0042, 'd20'), (0.0028, 'd12'), (0.0022, 'd13'), (0.0019, 'd6')]
idx:68
Idx: 12
idsTest: (1354,)
idx:1290
Idx: 13
idsTest: (1353,)
idx:1184
Idx: 14
idsTest: (1352,)
idx:652
Idx: 15
idsTest: (1351,)
idx:540
Idx: 16
idsTest: (1350,)
idx:946
Idx: 17
ids

100%|██████████| 1/1 [09:13<00:00, 553.63s/it]


In [34]:
databatch.to_frame().T

,ABS_f_D,DISS_wf10_D,STAT_e,STAT_n_D,STAT_n_A,STAT_CC_D,STAT_CC_A,STAT_CC_D_An,STAT_CC_A_Ca,ABS_wf_D,...,CT_f_conn_D_An,CT_f_conn_A_Ca,CT_e_conn,CT_e_D_An,CT_e_A_Ca,CT_f_D_tort1,CT_f_A_tort1,CT_n_D_adj_An,CT_n_A_adj_Ca,JSC
418,1.073907,2.357275,4.321095,1.073904,-1.073904,0.868869,6.370098,1.145694,5.500306,1.054819,...,1.290637,-1.513871,6.894226,6.965094,2.216067,-2.141377,-1.403761,0.463096,0.580252,3.679501


In [30]:
dataPool

,ABS_f_D,DISS_wf10_D,STAT_e,STAT_n_D,STAT_n_A,STAT_CC_D,STAT_CC_A,STAT_CC_D_An,STAT_CC_A_Ca,ABS_wf_D,...,CT_f_conn_D_An,CT_f_conn_A_Ca,CT_e_conn,CT_e_D_An,CT_e_A_Ca,CT_f_D_tort1,CT_f_A_tort1,CT_n_D_adj_An,CT_n_A_adj_Ca,JSC
170,1.481596,-0.444895,-0.522660,1.481603,-1.481603,-0.999492,1.903168,-1.481384,0.924866,1.419548,...,1.882087,-3.286218,-0.477342,1.149155,-1.307079,-1.428046,2.201719,0.579610,0.765183,0.289684
1628,-1.416170,0.160942,-0.094178,-1.416162,1.416162,0.264399,-0.724438,-0.167845,-0.790924,-1.373121,...,-0.779236,0.521964,-0.706060,-0.717943,0.110061,1.093861,-0.331322,0.463096,-0.080215,0.318276
1327,-0.872166,-0.401166,-0.439728,-0.872173,0.872173,-0.230167,-0.724438,-0.496230,-0.790924,-0.660919,...,-0.427587,0.521964,-0.548323,-0.628264,-0.238887,1.093861,0.596557,1.162181,1.478488,0.315967
1256,-1.198573,0.039940,-0.171063,-1.198567,1.198567,0.044592,-0.724438,-0.824615,-0.790924,-1.280495,...,-1.276394,0.521964,-1.310718,-1.050199,0.028777,1.093861,-0.824287,-2.053610,-0.793520,-1.077508
1286,-0.767478,0.730171,0.469070,-0.767470,0.767470,0.649062,-0.724438,0.817309,-0.790924,-0.774755,...,-0.813551,0.521964,-0.301202,-0.501830,0.660168,0.434726,-0.707634,-0.515623,-0.238727,0.259214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.264950,0.662647,0.425012,0.264939,-0.264939,0.154496,-0.461677,1.145694,-0.790924,0.286213,...,-0.777186,0.472372,-0.169755,-0.428322,0.552610,0.730024,-1.234022,1.395210,-0.080215,1.446512
1216,-0.379666,0.216062,0.009487,-0.379658,0.379658,0.154496,-0.461677,0.488924,-0.218994,-0.406605,...,-0.982134,0.521964,-0.603531,-0.665018,0.194630,0.744891,-0.471638,0.253371,-0.555752,0.362595
1653,-1.299755,0.510071,0.264331,-1.299760,1.299760,0.758965,-0.724438,0.488924,-0.790924,-1.258108,...,-0.895074,0.521964,-0.614047,-0.672368,0.471325,1.093861,-0.831673,1.045667,-1.004869,0.274757
559,0.121623,-1.000525,-0.870803,0.121630,-0.121630,-0.614829,-0.461677,-0.824615,-0.218994,0.194042,...,0.270427,0.521964,-0.201302,-0.425382,-0.674046,0.116443,0.419381,1.208787,-1.295475,-1.021155


In [38]:
1366-857

509